In [ ]:
#@title install miniconda
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
#@title Setup conda environment
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes
conda update -n base -c defaults conda --yes

In [ ]:
#@title Append python site packages to sys.path
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [ ]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Dependencies for feature extraction
!conda install -c conda-forge h5py -y
!conda install -c conda-forge opencv -y
import cv2

In [ ]:
#@title install dependencies for training
%%bash
pip install pytorch
pip install timm==0.4.12
pip install pickle5==0.0.9
pip install scikit-learn
pip install numpy

In [ ]:
#@title Initialize constant for files and paths
BASE_JSON_FILE = "/content/drive/MyDrive/MS-TCT-main/data/charades.json"
SLICED_JSON_FILE = "/content/drive/MyDrive/MS-TCT-main/data/charades_sliced.json"
PATH_TO_DATA_FOLDER = "/content/drive/MyDrive/MS-TCT-main/charades"

In [ ]:
#@title JSON slicing
import os
import shutil
import json

# Slice json 
# get current directory for charades.json file
src = BASE_JSON_FILE

# set directory for new charades.json file
dst = SLICED_JSON_FILE

# copy the file
path = shutil.copyfile(src,dst)

# get list of all extracted feature file names from feature_extract
dirList = os.listdir(PATH_TO_DATA_FOLDER)
dirList.remove(".DS_Store")
print(dirList)
# slicing chars 0 to 3 and last 4 to just have the file names
featList = []
for item in dirList:
  #item = item[4:]
  featList.append(item)

# modify the new file
## parse the new json file into dictionary to manage
f = open(dst)
data = json.load(f)
mod_data = {}
## find keys that are of the names in the featList inside of the data
for k in data:
  for name in featList:
    if k == name:
      mod_data[name] = data[k]

## overwrite this new data into the modified file
with open(dst, 'w') as f:
    json.dump(mod_data, f)

print(mod_data)

In [ ]:
#@title Feature extraction
#feature extraction, 
#input: frames , output: npy files
!python3 /content/drive/MyDrive/MS-TCT-main/pytorch-i3d/extract_features.py \
-mode rgb \
-load_model /content/drive/MyDrive/MS-TCT-main/pytorch-i3d/models/rgb_charades.pt \
-root /content/drive/MyDrive/MS-TCT-main/charades \
-gpu 0\
-save_dir /content/drive/MyDrive/MS-TCT-main/feature_extract

#training

In [ ]:
#@title model training - MS-TCT
!python3 /content/drive/MyDrive/MS-TCT-main/train.py \
-dataset charades\
-mode rgb \
-model MS_TCT\
-train True \
-num_clips 256 \
-skip 0 \
-lr 0.0001 \
-comp_info False \
-epoch 50 \
-unisize True \
-alpha_l 1 \
-beta_l 0.05 \
-batch_size 32 


#evaluation


In [ ]:
#@title evaluation
!python3 "/content/drive/MyDrive/MS-TCT-main/Evaluation.py"\
-pkl_path "/content/drive/MyDrive/MS-TCT-main/save_logit/0.pkl"